# Conversion to nfiti data from bruker 

In [ ]:
# Setting the data folder and the data point
DATA_FOLDER='/home/smummaneni/nbrc/data/new/Raw_data_MRS'
DATA_POINT='118W8_09'
FILE_OUT='FID_' + DATA_POINT + '.nii.gz'
OUTPUT_FOLDER='/home/smummaneni/nbrc/data/new/Raw_data_MRS/proc'
BASIS_FILE='/home/smummaneni/nbrc/data/new/9.4t/gamma_press_te15_9.4t_v1.basis'
!mkdir OUTPUT_FOLDER
# Run spec2nii to convert FID to NIfTI
!spec2nii bruker -m FID $DATA_FOLDER/$DATA_POINT
# The output will in current directory, move it to the data folder
!mv $FILE_OUT $DATA_FOLDER/.

## Processing - fixed phase (0 order, 90 deg) --> apodize (exp 25) --> remove --> model (? [1.5-4.0])

In [31]:
# Applied sperately to the FID data
%sx fsl_mrs_proc remove --file $DATA_FOLDER/$FILE_OUT --out $OUTPUT_FOLDER --filename removed
%sx fsl_mrs_proc apodize --file $DATA_FOLDER/$FILE_OUT --out $OUTPUT_FOLDER --filename smoothed --filter exp --amount 25
%sx fsl_mrs_proc fixed_phase --file $DATA_FOLDER/$FILE_OUT --out $OUTPUT_FOLDER --filename phase_corrected --p0 90
%sx fsl_mrs_proc model --file $DATA_FOLDER/$FILE_OUT --out $OUTPUT_FOLDER --filename peak --ppm 1.5 4.0

# Combine processing
%sx fsl_mrs_proc fixed_phase --file $DATA_FOLDER/$FILE_OUT --out $OUTPUT_FOLDER --filename fully_processed --p0 90
%sx fsl_mrs_proc apodize --file $OUTPUT_FOLDER/fully_processed.nii.gz --out $OUTPUT_FOLDER --filename fully_processed --filter exp --amount 25
%sx fsl_mrs_proc remove --file $OUTPUT_FOLDER/fully_processed.nii.gz --out $OUTPUT_FOLDER --filename fully_processed
%sx fsl_mrs_proc model --file $OUTPUT_FOLDER/fully_processed.nii.gz --out $OUTPUT_FOLDER --filename peak_filtered --ppm 1.5 4.0


[]

## Fitting

In [34]:
%sx fsl_mrs --data $OUTPUT_FOLDER/fully_processed.nii.gz --basis $BASIS_FILE --output out

[]

In [36]:
data = 'FID_118W8_09.nii.gz'
out_dir = 'proc'
print('For individual processing')
print('fsl_mrs_proc remove --file ' + data + ' --out ' + out_dir + ' --filename removed')
print('fsl_mrs_proc apodize --file ' + data + ' --out ' + out_dir + ' --filename smoothed --filter exp --amount 25')
print('fsl_mrs_proc fixed_phase --file ' + data + ' --out ' + out_dir + ' --filename phase_corrected --p0 90')
print('fsl_mrs_proc model --file ' + data + ' --out ' + out_dir + ' --filename peak --ppm 1.5 4.0')
print('-------------------------------------------------')
print('For combined processing')
print('fsl_mrs_proc fixed_phase --file ' + data + ' --out ' + out_dir + ' --filename fully_processed --p0 90')
print('fsl_mrs_proc apodize --file ' + out_dir + '/fully_processed.nii.gz --out ' + out_dir + ' --filename fully_processed --filter exp --amount 25')
print('fsl_mrs_proc remove --file ' + out_dir + '/fully_processed.nii.gz --out ' + out_dir + ' --filename fully_processed')
print('fsl_mrs_proc model --file ' + out_dir + '/fully_processed.nii.gz --out ' + out_dir + ' --filename peak_filtered --ppm 1.5 4.0')
print('-------------------------------------------------')
print('Fitting')
print('fsl_mrs --data ' + out_dir + '/fully_processed.nii.gz --basis ' + BASIS_FILE + ' --output out')

For individual processing
fsl_mrs_proc remove --file FID_118W8_09.nii.gz --out proc --filename removed
fsl_mrs_proc apodize --file FID_118W8_09.nii.gz --out proc --filename smoothed --filter exp --amount 25
fsl_mrs_proc fixed_phase --file FID_118W8_09.nii.gz --out proc --filename phase_corrected --p0 90
fsl_mrs_proc model --file FID_118W8_09.nii.gz --out proc --filename peak --ppm 1.5 4.0
-------------------------------------------------
For combined processing
fsl_mrs_proc fixed_phase --file FID_118W8_09.nii.gz --out proc --filename fully_processed --p0 90
fsl_mrs_proc apodize --file proc/fully_processed.nii.gz --out proc --filename fully_processed --filter exp --amount 25
fsl_mrs_proc remove --file proc/fully_processed.nii.gz --out proc --filename fully_processed
fsl_mrs_proc model --file proc/fully_processed.nii.gz --out proc --filename peak_filtered --ppm 1.5 4.0
-------------------------------------------------
Fitting
fsl_mrs --data proc/fully_processed.nii.gz --basis /home/smumm